In [8]:
import torch
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image, ImageDraw
import cv2 as cv
import random

In [9]:
from pymoo.core.problem import Problem
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation
from pymoo.optimize import minimize
from pymoo.core.sampling import Sampling

import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr

In [10]:
from config_recons import DATA, MODEL, COLOR_INDEX, PRECOMPUTED_COLORS

In [11]:
def convert_to_3d(flattened_patches, patch_size=20):
    if flattened_patches.shape[0] == 1:
        return flattened_patches[0].reshape(patch_size, patch_size, 3)
    return np.array([patch.reshape(patch_size, patch_size, 3) for patch in flattened_patches])

In [12]:
class Fitness:
    def __init__(self, location, model, img1, img2, label):
        self.img1 = img1
        self.img2 = img2
        self.model = model
        self.label = label
        self.location = location
        
        self.original_patch = self.take_patch_from_image(img1, location)
        img2_torch = transforms.ToTensor()(Image.fromarray(img2)).unsqueeze(0).cuda()
        self.img2_feature = model(img2_torch)
    
    def take_patch_from_image(self, img1, location):
        x_min, x_max, y_min, y_max = location
        patch = img1[y_min:y_max, x_min:x_max, :]
        return patch.astype('uint8')
    
    def evaluate_psnr(self, patchs): # np all
        patchs_3d = convert_to_3d(patchs)

        r_psnr = np.array([psnr(p[:,:,0], self.original_patch[:,:,0]) for p in patchs_3d])
        g_psnr = np.array([psnr(p[:,:,1], self.original_patch[:,:,1]) for p in patchs_3d])
        b_psnr = np.array([psnr(p[:,:,2], self.original_patch[:,:,2]) for p in patchs_3d])

        psnr_score = (r_psnr + g_psnr + b_psnr) / 3
        
        return psnr_score / 40 

    def apply_patch_to_image(self, patch):
        patchs_3d = convert_to_3d(np.array([patch]))
        img_copy = self.img1.copy()
        x_min, x_max, y_min, y_max = self.location
        
        img_copy[y_min : y_max, x_min : x_max, :] = patchs_3d.astype('uint8')
        return img_copy

    def evaluate_adv(self, patchs, threshold=0.5, transform=transforms.ToTensor()):
        patchs_3d = convert_to_3d(patchs)

        adv_imgs = [self.apply_patch_to_image(p) for p in patchs_3d]
        
        
        with torch.no_grad():
            adv_batch = torch.stack([transform(Image.fromarray(img)) for img in adv_imgs]).cuda()

            adv_features = self.model(adv_batch)
            
            sims = F.cosine_similarity(adv_features, self.img2_feature, dim=1)
            adv_scores = torch.zeros_like(sims).cuda()

            adv_scores = (1 - self.label) * (threshold - sims) + self.label * (sims - threshold)
            
            return adv_scores.cpu().numpy()
        
    def benchmark(self, patchs):
        adv_scores = - self.evaluate_adv(patchs)
        fsnr_scores = - self.evaluate_psnr(patchs)
        
        return [adv_scores, fsnr_scores]


In [21]:
img1, img2, label = DATA[0]
img1, img2 = img1.resize((160, 160)), img2.resize((160, 160))
img1_np, img2_np = np.array(img1), np.array(img2)

location, (w, h) = (90, 110, 80, 100), (20, 20)


In [22]:
fit = Fitness(location, MODEL, img1_np, img2_np, label)

In [23]:
class PatchFaceAttack(Problem):
    def _evaluate(self, designs, out, *args, **kwargs):
        res = fit.benchmark(designs)
        out['F'] = np.array(res)        

In [24]:
def get_next_color():
    color = PRECOMPUTED_COLORS[COLOR_INDEX]
    COLOR_INDEX = (COLOR_INDEX + 1) % len(PRECOMPUTED_COLORS)
    return color

def add_random_shape_to_image(patch, number_of_shapes):
    patch_3d = convert_to_3d(np.array([patch]))
    image_with_shapes = patch_3d.copy()
    height, width = patch_3d.shape[:2]

    shape_types = ['circle', 'rectangle', 'ellipse', 'line']

    for _ in range(number_of_shapes):
        shape_type = random.choice(shape_types)

        color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        )
        thickness = random.randint(1, 5)

        if shape_type == 'circle':
            center = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            radius = random.randint(5, min(width, height) // 4)
            cv.circle(image_with_shapes, center, radius, color, thickness)

        elif shape_type == 'rectangle':
            pt1 = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            pt2 = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            cv.rectangle(image_with_shapes, pt1, pt2, color, thickness)

        elif shape_type == 'ellipse':
            center = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            axes = (
                random.randint(5, width // 4),
                random.randint(5, height // 4)
            )
            angle = random.randint(0, 360)
            start_angle = 0
            end_angle = 360
            cv.ellipse(image_with_shapes, center, axes, angle, start_angle, end_angle, color, thickness)

        elif shape_type == 'line':
            pt1 = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            pt2 = (
                random.randint(0, width - 1),
                random.randint(0, height - 1)
            )
            cv.line(image_with_shapes, pt1, pt2, color, thickness)

    return image_with_shapes
        
def create_random_population(size, patch_size=20):
    patchs = [np.zeros(patch_size**2 * 3, dtype=np.uint8) for _ in range(size)]
    
    return np.array([add_random_shape_to_image(patch, 2).flatten() for patch in patchs])



In [25]:
from pymoo.core.population import Population

class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):

        patchs = create_random_population(n_samples)
        return patchs


In [26]:
import random
import numpy as np
from PIL import Image

def random_horizontal_swap(patch1, patch2):
    offspring = patch1.copy()
    height, width = patch1.shape[:2]
    horizontal_random_choice = np.random.choice(width, int(width / 2), replace=False)
    offspring[horizontal_random_choice] = patch2[horizontal_random_choice]
    return offspring.astype('uint8')

def random_vertical_swap(patch1, patch2):
    offspring = patch1.copy()
    height, width = patch1.shape[:2]
    vertical_random_choice = np.random.choice(height, int(height / 2), replace=False)
    offspring[:, vertical_random_choice] = patch2[:, vertical_random_choice]
    return offspring.astype('uint8')

def cut_and_merge(patch1, patch2):
    offspring = patch1.copy()
    height, width = patch1.shape[:2]
    x_cut = random.randint(0, width - 1)
    y_cut = random.randint(0, height - 1) 
    offspring[y_cut:, x_cut:] = patch2[y_cut:, x_cut:]
    return offspring.astype('uint8')

def single_point_crossover(patch1, patch2):
    offspring = patch1.copy()
    height, width = patch1.shape[:2]
    cut_point = random.randint(0, height - 1)
    offspring[:cut_point, :] = patch2[:cut_point, :]
    return offspring.astype('uint8')

def two_point_crossover(patch1, patch2):
    offspring = patch1.copy()
    height, width = patch1.shape[:2]
    cut_point1 = random.randint(0, height // 2)
    cut_point2 = random.randint(cut_point1, height)  
    offspring[cut_point1:cut_point2, :] = patch2[cut_point1:cut_point2, :]
    return offspring.astype('uint8')

class BinaryCrossover(Crossover):
    def __init__(self):
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        pop1 = X[0, :, :]  # n_samples x 1200
        pop2 = X[1, :, :]

        print("X: ", X.shape)
        
        n_parents, n_offsprings = self.n_parents, self.n_offsprings
        n_matings = 2 
        
        offspring = np.zeros((n_offsprings, n_matings, problem.n_var))  # 1, 2, 1200

        for i in range(n_matings):
            patch1_index = np.random.choice(pop1.shape[0])
            patch2_index = np.random.choice(pop2.shape[0])

            patch1 = pop1[patch1_index].reshape(1, -1)
            patch2 = pop2[patch2_index].reshape(1, -1)
            patch1, patch2 = convert_to_3d(patch1), convert_to_3d(patch2)

            crossover_type = random.choice(['horizontal', 'vertical', 'cut_merge', 'single_point', 'two_point'])

            if crossover_type == 'horizontal':
                result = random_horizontal_swap(patch1, patch2)
            elif crossover_type == 'vertical':
                result = random_vertical_swap(patch1, patch2)
            elif crossover_type == 'cut_merge':
                result = cut_and_merge(patch1, patch2)
            elif crossover_type == 'single_point':
                result = single_point_crossover(patch1, patch2)
            elif crossover_type == 'two_point':
                result = two_point_crossover(patch1, patch2)

            result = result.reshape(1, -1)  

            offspring[i, 0, :] = result 

        return offspring



In [27]:
def mutate(patch):
    return add_random_shape_to_image(patch)

class MyMutation(Mutation):
    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            X[i] = mutate(X[i])

        return X

In [28]:
from pymoo.algorithms.moo.nsga2 import NSGA2

problem = PatchFaceAttack(n_var=1200, n_obj=2, xl=-255, xu=255)
sampling = MySampling()
mutation = MyMutation()
crossover = BinaryCrossover()

algorithm = NSGA2(pop_size=5,
                  sampling=sampling,
                  crossover=BinaryCrossover(),
                  mutation=mutation)
stop_criteria = ('n_gen', 100)

In [29]:
results = minimize(
    problem=problem,
    algorithm=algorithm,
    termiation=stop_criteria,
    seed=1,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        5 |      1 |             - |             -
X:  (2, 5, 1200)


IndexError: index 1 is out of bounds for axis 0 with size 1